# NLP Getting Started (Kaggle Competition)

https://www.kaggle.com/c/nlp-getting-started - this notebook placed 488/809 at 79%

The task was to classify tweets as to whether they were about a disaster or not 

File requirements - train.csv and test.csv from above link, and GoogleNews-vectors-negative300.bin.gz - accessible online e.g. https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g or searching Google News Word2Vec - it's publically available

A lot of the below packages are probably not needed but I wasn't bothered to go through and delete them

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence,to_categorical

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import random
import re
import pickle
import os
from collections import Counter
import time
from tqdm import tqdm
from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence,to_categorical

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import random
import re
import pickle
import os
from collections import Counter
import time
from tqdm import tqdm
from gensim.models import Word2Vec

tf.keras.backend.clear_session()
tqdm.pandas()
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
import itertools
tf.keras.backend.clear_session()
tqdm.pandas()
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [2]:
data = pd.read_csv('train.csv')
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Clean text - remove punctuation and stopwords, and stem (lemmatization, e.g. stopped --> stop)

In [3]:
import re
data['text'] = data['text'].str.lower()
data['text']  = data['text'].astype(str)
data['text'] = data['text'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
data['text'] = data['text'].str.replace('nan','')
data


,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are ...,1
3,6,NaN,NaN,people receive wildfires evacuation or...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,aria ahrary thetawniest the out of control w...,1
7610,10871,NaN,NaN,m utc km s of volcano hawaii htt...,1
7611,10872,NaN,NaN,police investigating after an e bike collided ...,1


In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import nltk
stemmer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

for i in range(len(data['text'])):
    lemm = [stemmer.lemmatize(word, get_wordnet_pos(word)) for word in data['text'][i].split()] 
    data['text'][i] =' '.join(lemm)
data

C:\Users\eparrag\AppData\Local\Continuum\anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,keyword,location,text,target
0,1,NaN,NaN,our deed be the reason of this earthquake may ...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all resident ask to shelter in place be be not...,1
3,6,NaN,NaN,people receive wildfire evacuation order in ca...,1
4,7,NaN,NaN,just get sent this photo from ruby alaska a sm...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant crane hold a bridge collapse into ne...,1
7609,10870,NaN,NaN,aria ahrary thetawniest the out of control wil...,1
7610,10871,NaN,NaN,m utc km s of volcano hawaii http t co zdtoyd ebj,1
7611,10872,NaN,NaN,police investigate after an e bike collide wit...,1


In [5]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]
for i in range(len(data['text'])):
    stopped_words = remove_stop_words(data['text'][i].split())
    data['text'][i] =' '.join(stopped_words)
data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eparrag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\eparrag\AppData\Local\Continuum\anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake may allah forgive u,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,resident ask shelter place notify officer evac...,1
3,6,NaN,NaN,people receive wildfire evacuation order calif...,1
4,7,NaN,NaN,get sent photo ruby alaska smoke wildfire pour...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant crane hold bridge collapse nearby ho...,1
7609,10870,NaN,NaN,aria ahrary thetawniest control wild fire cali...,1
7610,10871,NaN,NaN,utc km volcano hawaii http co zdtoyd ebj,1
7611,10872,NaN,NaN,police investigate e bike collide car little p...,1


In [6]:
from sklearn.utils import shuffle
data = shuffle(data).reset_index()
data

,index,id,keyword,location,text,target
0,6792,9731,tragedy,NaN,rly tragedy mp live recount horror saw coach t...,1
1,4441,6320,hostage,"Melbourne, FL",wtf biological father hold hostage adoptive pa...,0
2,1712,2468,collided,"St. Joseph, Minnesota",two train collide india please pray victim fam...,1
3,3452,4936,exploded,NaN,reason bluewestlo explode youtube http co upgd...,0
4,7297,10443,wild%20fires,Eastern Iowa,several wild fire burn lot land california one...,1
...,...,...,...,...,...,...
7608,6322,9038,stretcher,Florida,restore vinyl side make look new http co rdxzs...,0
7609,6631,9496,terrorist,NaN,signing petition seek mercy death punishment c...,1
7610,3766,5349,fire,"Vancouver, BC",justin ling promise tax pancake rainbow die fire,0
7611,6428,9193,suicide%20bombing,NaN,suicide bombing suru kill people turkey launch...,1


In [7]:
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stoplist = stopwords.words('english')
from autocorrect import Speller
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

print(data['text'])
spell = Speller(lang='en')
     
#data['ServiceTextValue'] = result
print('tokenizing')
tokens = [word_tokenize(sentence) for sentence in data.text]
data['tokens'] = tokens #stopped_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eparrag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       rly tragedy mp live recount horror saw coach t...
1       wtf biological father hold hostage adoptive pa...
2       two train collide india please pray victim fam...
3       reason bluewestlo explode youtube http co upgd...
4       several wild fire burn lot land california one...
                              ...                        
7608    restore vinyl side make look new http co rdxzs...
7609    signing petition seek mercy death punishment c...
7610     justin ling promise tax pancake rainbow die fire
7611    suicide bombing suru kill people turkey launch...
7612    gotta love summer calgary yyc hailstorm crazyw...
Name: text, Length: 7613, dtype: object
tokenizing


In [8]:
all_training_words = [word for tokens in data["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data["tokens"].tolist())
training_sequences = tokenizer.texts_to_sequences(data["tokens"].tolist())
import pickle
with open('tokenizer.pk', 'wb') as fout:
        pickle.dump(tokenizer, fout)

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
print("Max sentence length is %s" % max(training_sentence_lengths))

Found 19768 unique tokens.
Max sentence length is 27


In [9]:
train_rnn_data = pad_sequences(training_sequences, maxlen=30)

In [10]:
print(train_rnn_data.shape)


(7613, 30)


In [50]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [52]:
EMBEDDING_SIZE = 300
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_SIZE))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_SIZE)
    #train_embedding_weights[index,:] = word2vec.wv[word] if word in word2vec.wv else np.random.rand(EMBEDDING_SIZE)
print(train_embedding_weights.shape)

(19769, 300)


In [78]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)


    lstm = LSTM(128)(embedded_sequences)
    x = Dense(64, activation='relu')(lstm)
    x = Dropout(0.4)(x)

    preds = Dense(labels_index, activation='sigmoid')(x)
    #opt = tf.keras.optimizers.Adam(lr=0.01,clipnorm=1.0)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model
print('done')

done


In [79]:
y_train = data['target'].values
x_train = train_rnn_data

model = recurrent_nn(train_embedding_weights, 30, len(train_word_index)+1, 300, 
                    1)
print('training model....')

num_epochs = 6
batch_size = 128
hist = model.fit(x_train, y_train, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 30, 300)           5930700   
_________________________________________________________________
lstm_21 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_54 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_23 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 65        
Total params: 6,158,669
Trainable params: 227,969
Non-trainable params: 5,930,700
__________________________________________

In [64]:
model.save('RNN_lstm.h5')